The aim of this exercise is to train you in debugging networks using the good old print function and also tensorboard. To simulate poor training, we will train a multilayer perceptron using the CIFAR data.

1. Use the CIFAR data set reader from the first homework and read the CIFAR-10 files again. 
2. Apply random noise to the image 
3. Convert the image to float and scale to [0.0, 1.0] by dividing the pixel values by the highest pixel value.
4. Convert all labels to onehot encoding
5. Build a 3-layer multilayer perceptron of size [512, 256, 128]. 
6. Create a tensorboard summary for plotting the histogram of the weights of the three layers.
7. Also write the cost / loss at the end of each epoch to tensorboard.
8. Train the network with learning rates of [0.1, 0.01, 0.001]. You will notice that the network will not converge well.
9. Submit the snapshot of the histograms for the three learning rates. Describe your observations.



In [20]:
import pickle
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import pandas as pd 
import random
import datetime

random.seed(3457)

#1. Use the CIFAR data set reader from the first homework and read the CIFAR-10 files again. 
files = glob.glob(
    "/Users/omerorhan/Desktop/UCSC/Deep Learning/cifar-10-batches-py/data_batch*")

def unpickle(file, types):
    with open(file, 'rb') as fo:
        return pickle.load(fo, encoding='bytes').get(types)

data = np.concatenate([unpickle(x, b'data') for x in files])
labels = np.concatenate([unpickle(x, b'labels') for x in files])




In [12]:
#2. Apply random noise to the image 
mean = 1   # some constant
std = 1    # some constant (standard deviation)
noisy_data = data + np.random.normal(mean, std, data.shape)

#3. Convert the image to float and scale to [0.0, 1.0] by dividing the pixel values by the highest pixel value.
noisy_data = noisy_data.astype(float)
noisy_data = np.divide(noisy_data, np.amax(noisy_data[0]))

noisy_data = np.float32(noisy_data)


In [21]:
#4. Convert all labels to onehot encoding
n_values = np.max(labels) + 1
print(type(labels))
labelsencoded = np.eye(n_values)[labels]
labelsencoded = np.float32(labelsencoded)

print(labelsencoded[25])

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [22]:
# Hyper-parameters

learningrate = 0.1
nepochs = 10
batch_size = 128
#128,3072
nhidden1 = 512
nhidden2 = 256 
nhidden3 = 128 
ninput = noisy_data[0].flatten().shape[0]
noutput = 10

X = tf.placeholder(tf.float32, [batch_size,ninput])
Y = tf.placeholder(tf.float32, [batch_size, noutput])


print(X)
print(Y)

Tensor("Placeholder_6:0", shape=(128, 3072), dtype=float32)
Tensor("Placeholder_7:0", shape=(128, 10), dtype=float32)


In [23]:
#5. Build a 3-layer multilayer perceptron of size [512, 256, 128]. 

weights = \
{
        'h1': tf.Variable(tf.random_normal([ninput, nhidden1])),
        'h2': tf.Variable(tf.random_normal([nhidden1, nhidden2])),
        'h3': tf.Variable(tf.random_normal([nhidden2, nhidden3])),
        'out': tf.Variable(tf.random_normal([nhidden3, noutput]))
}

biases = \
{
    'b1': tf.Variable(tf.random_normal([nhidden1])),
    'b2': tf.Variable(tf.random_normal([nhidden2])),
    'b3': tf.Variable(tf.random_normal([nhidden3])),
    'out': tf.Variable(tf.random_normal([noutput]))
}

def multiperceptron(x):
    l1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    l2 = tf.nn.sigmoid(tf.add(tf.matmul(l1, weights['h2']), biases['b2']))
    l3 = tf.nn.sigmoid(tf.add(tf.matmul(l2, weights['h3']), biases['b3']))
    outl = tf.nn.sigmoid(tf.add(tf.matmul(l3, weights['out']), biases['out']))
    return outl

model = multiperceptron(X)


Tensor("random_normal_32:0", shape=(3072, 512), dtype=float32)


Tensor("random_normal_33:0", shape=(512, 256), dtype=float32)
Tensor("random_normal_34:0", shape=(256, 128), dtype=float32)
Tensor("random_normal_35:0", shape=(128, 10), dtype=float32)


In [24]:

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
train_min = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
# argmax tells what index is the max in the y hat
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#6. Create a tensorboard summary for plotting the histogram of the weights of the three layers.
# Add to tensorboard summary
tf.summary.histogram("weight_1",weights['h1'])
tf.summary.histogram("weight_2",weights['h2'])
tf.summary.histogram("weight_3",weights['h3'])
tf.summary.scalar("loss", tf.constant(12))

merged_summary_op = tf.summary.merge_all()
filename="/Users/omerorhan/Desktop/UCSC/Deep Learning/DeepLearningUCSC/venv/homework/Assignments" \
         "/logs/run"+datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
writer = tf.summary.FileWriter(filename, tf.get_default_graph())
init = tf.global_variables_initializer()


In [25]:
import random
def getbatch(alldata, alllabels, batch_size = 16):
    nlabels = len(alllabels)
    number_of_batches = nlabels//batch_size # TODO: Change 100 to nlabels
    for batch_number in range(number_of_batches):
        rand_index = [random.randrange(0, nlabels) for i in range(batch_size)]
        batch_x = alldata[rand_index]
        batch_y = np.array([alllabels[idx] for idx in rand_index])
        yield (batch_x, batch_y)
 


In [26]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        for batch_x, batch_y in getbatch(noisy_data, labelsencoded, batch_size):
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y})
        losscalc, accuracycalc, merged_summary = \
                      sess.run([loss, accuracy, merged_summary_op], feed_dict={X:batch_x, Y:batch_y})
        writer.add_summary(merged_summary, epoch) 
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
writer.close()

Epoch: 0, Loss: 2.3026, Accuracy: 0.0859


Epoch: 1, Loss: 2.3026, Accuracy: 0.0469


Epoch: 2, Loss: 2.3026, Accuracy: 0.1016


Epoch: 3, Loss: 2.3026, Accuracy: 0.1172


Epoch: 4, Loss: 2.3026, Accuracy: 0.1016


Epoch: 5, Loss: 2.3026, Accuracy: 0.0547


Epoch: 6, Loss: 2.3026, Accuracy: 0.0703


Epoch: 7, Loss: 2.3026, Accuracy: 0.1250


Epoch: 8, Loss: 2.3026, Accuracy: 0.0625


Epoch: 9, Loss: 2.3026, Accuracy: 0.0859


In [ ]:
"""
learning_rate= 0.1
Epoch: 0, Loss: 2.3026, Accuracy: 0.0859
Epoch: 1, Loss: 2.3026, Accuracy: 0.0469
Epoch: 2, Loss: 2.3026, Accuracy: 0.1016
Epoch: 3, Loss: 2.3026, Accuracy: 0.1172
Epoch: 4, Loss: 2.3026, Accuracy: 0.1016
Epoch: 5, Loss: 2.3026, Accuracy: 0.0547
Epoch: 6, Loss: 2.3026, Accuracy: 0.0703
Epoch: 7, Loss: 2.3026, Accuracy: 0.1250
Epoch: 8, Loss: 2.3026, Accuracy: 0.0625
Epoch: 9, Loss: 2.3026, Accuracy: 0.0859
"""